In [150]:
import pandas as pd
import matplotlib.pyplot as plt

In [151]:
mwobs = pd.read_csv('MtWashington_Observations.csv')

C:\Users\benja\AppData\Local\Temp\ipykernel_26524\502386398.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  mwobs = pd.read_csv('MtWashington_Observations.csv')


In [152]:
mwobs['valid'] = pd.to_datetime(mwobs['valid'])

In [153]:
mwobs.columns

Index(['station', 'valid', 'tmpf', 'dwpf', 'relh', 'drct', 'sknt', 'p01i',
       'alti', 'mslp', 'vsby', 'gust', 'skyc1', 'skyc2', 'skyc3', 'skyc4',
       'skyl1', 'skyl2', 'skyl3', 'skyl4', 'wxcodes', 'ice_accretion_1hr',
       'ice_accretion_3hr', 'ice_accretion_6hr', 'peak_wind_gust',
       'peak_wind_drct', 'peak_wind_time', 'feel', 'metar', 'snowdepth'],
      dtype='object')

In [154]:
condition = ((mwobs['valid'].dt.hour == 5) & (mwobs['valid'].dt.minute <= 30)) | \
            ((mwobs['valid'].dt.hour == 4) & (mwobs['valid'].dt.minute >= 30)) | \
            ((mwobs['valid'].dt.hour == 15) & (mwobs['valid'].dt.minute <= 30)) | \
            ((mwobs['valid'].dt.hour == 14) & (mwobs['valid'].dt.minute >= 30)) | \
            ((mwobs['valid'].dt.hour == 22) & (mwobs['valid'].dt.minute >= 30)) | \
            ((mwobs['valid'].dt.hour == 23) & (mwobs['valid'].dt.minute <= 30))

filtered_mwobs = mwobs[condition]

In [155]:
mwobs_init = filtered_mwobs.drop(columns = ['station', 'metar', 'skyc1', 'skyc2', 'skyc3', 'skyc4',
       'skyl1', 'skyl2', 'skyl3', 'skyl4', 'ice_accretion_1hr',
       'ice_accretion_3hr', 'peak_wind_time', 'feel', 'metar', 'snowdepth', 'peak_wind_gust', 'peak_wind_drct', 'alti', 'mslp', 'ice_accretion_6hr', 'p01i'])

In [156]:
mwobs_init = mwobs_init.drop(mwobs_init['tmpf'][mwobs_init['tmpf'] == 'M'].index)

In [157]:
mwobs_init.loc[mwobs_init['gust'] == 'M', 'gust'] = mwobs_init['sknt']

In [158]:
mwobs_init[mwobs_init['wxcodes'] == '#NAME?']

,valid,tmpf,dwpf,relh,drct,sknt,vsby,gust,wxcodes
98,2014-07-05 04:49:00,39.2,39.2,100,360,31,0,44,#NAME?
217,2014-07-10 04:49:00,50,50,100,280,39,0,44,#NAME?
306,2014-07-13 22:49:00,53.6,53.6,100,250,35,0,46,#NAME?
358,2014-07-16 04:48:00,51.8,51.8,100,260,27,0.06,27,#NAME?
368,2014-07-16 14:52:00,53.6,53.6,100,260,12,0.06,12,#NAME?
...,...,...,...,...,...,...,...,...,...
84496,2024-04-19 22:52:00,33.8,32,93.03,230,27,0.06,43,#NAME?
84512,2024-04-20 14:47:00,26.6,26.6,100,270,27,0,27,#NAME?
84520,2024-04-20 22:53:00,23,23,100,270,37,0.06,37,#NAME?
84544,2024-04-21 22:53:00,17.6,17.6,100,280,35,0.06,40,#NAME?


In [159]:
mwobs_init['wxcodes'].value_counts()[0:15]

wxcodes
M               4098
#NAME?          2257
FG              1794
FZFG            1116
FZFG BLSN        483
BCFG             209
PRFG              82
RA FG             66
SN FZFG BLSN      61
FZFG DRSN         54
SHRA FG           43
DRSN              27
DZ FG             27
VCSH              26
SN FZFG DRSN      22
Name: count, dtype: int64

In [160]:
mwobs_init.insert(1, 'SN', mwobs_init['wxcodes'].str.contains('SN'))

In [161]:
mwobs_init.insert(1, 'RA', mwobs_init['wxcodes'].str.contains('RA'))
mwobs_init.insert(1, 'FG', mwobs_init['wxcodes'].str.contains('FG'))
mwobs_init.insert(1, 'BL', mwobs_init['wxcodes'].str.contains('BL'))

In [162]:
mwobs_init.insert(1, 'M', mwobs_init['wxcodes'].str.contains('M'))
mwobs_init.insert(1, 'UP', mwobs_init['wxcodes'].str.contains('#NAME?'))

In [163]:
mwobs_init = mwobs_init.drop(columns = 'wxcodes')

In [164]:
mwobs_init

,valid,UP,M,BL,FG,RA,SN,tmpf,dwpf,relh,drct,sknt,vsby,gust
4,2014-07-01 04:49:00,False,False,False,True,False,False,55.4,55.4,100,270,28,0,28
21,2014-07-01 22:57:00,False,True,False,False,False,False,62.6,55.4,77.29,240,37,10,41
27,2014-07-02 04:50:00,False,False,False,True,False,False,57.2,57.2,100,250,42,0,49
36,2014-07-02 14:52:00,False,True,False,False,False,False,60.8,57.2,87.92,270,21,40,21
44,2014-07-02 22:54:00,False,True,False,False,False,False,57.2,53.6,87.74,240,20,50,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84631,2024-04-25 14:48:00,False,True,False,False,False,False,10.4,-4,51.41,330,39,100,45
84639,2024-04-25 22:53:00,False,True,False,False,False,False,15.8,8.6,72.73,300,31,110,37
84645,2024-04-26 04:54:00,False,True,False,False,False,False,14,6.8,72.53,340,23,120,23
84655,2024-04-26 14:53:00,False,True,False,False,False,False,17.6,1.4,48.39,340,9,100,9


In [165]:
mwobs_init.drop((mwobs_init[mwobs_init['drct'] == 'M']).index, inplace=True)

In [167]:
mwobs_init.reset_index(inplace=True)

In [168]:
mwobs_init

,index,valid,UP,M,BL,FG,RA,SN,tmpf,dwpf,relh,drct,sknt,vsby,gust
0,4,2014-07-01 04:49:00,False,False,False,True,False,False,55.4,55.4,100,270,28,0,28
1,21,2014-07-01 22:57:00,False,True,False,False,False,False,62.6,55.4,77.29,240,37,10,41
2,27,2014-07-02 04:50:00,False,False,False,True,False,False,57.2,57.2,100,250,42,0,49
3,36,2014-07-02 14:52:00,False,True,False,False,False,False,60.8,57.2,87.92,270,21,40,21
4,44,2014-07-02 22:54:00,False,True,False,False,False,False,57.2,53.6,87.74,240,20,50,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10517,84631,2024-04-25 14:48:00,False,True,False,False,False,False,10.4,-4,51.41,330,39,100,45
10518,84639,2024-04-25 22:53:00,False,True,False,False,False,False,15.8,8.6,72.73,300,31,110,37
10519,84645,2024-04-26 04:54:00,False,True,False,False,False,False,14,6.8,72.53,340,23,120,23
10520,84655,2024-04-26 14:53:00,False,True,False,False,False,False,17.6,1.4,48.39,340,9,100,9


In [ ]:
wdirs = mwobs_init['drct'][:]
wdirs = wdirs.astype(int)
wdirs = np.array(wdirs)

wdir_vec = [0, 45, 90, 135, 180, 225, 270, 315, 360]
wdir_cats = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', 'N']

new_wd = []
for i in range(len(wdirs)):
  wdir_obs_vec = [wdirs[i] for j in range(len(wdir_vec))]
  idxmin = np.argmin(np.abs(np.array(wdir_obs_vec) - np.array(wdir_vec)))
  new_wd.append(wdir_cats[idxmin])

new_wd

In [170]:
mwobs_init.insert(1, 'WindDir', new_wd)

In [172]:
mwobs_init.drop(columns = 'drct', inplace=True)

In [174]:
mwobs_init.drop(columns = 'index', inplace=True)
mwobs_init

,WindDir,valid,UP,M,BL,FG,RA,SN,tmpf,dwpf,relh,sknt,vsby,gust
0,W,2014-07-01 04:49:00,False,False,False,True,False,False,55.4,55.4,100,28,0,28
1,SW,2014-07-01 22:57:00,False,True,False,False,False,False,62.6,55.4,77.29,37,10,41
2,W,2014-07-02 04:50:00,False,False,False,True,False,False,57.2,57.2,100,42,0,49
3,W,2014-07-02 14:52:00,False,True,False,False,False,False,60.8,57.2,87.92,21,40,21
4,SW,2014-07-02 22:54:00,False,True,False,False,False,False,57.2,53.6,87.74,20,50,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10517,NW,2024-04-25 14:48:00,False,True,False,False,False,False,10.4,-4,51.41,39,100,45
10518,NW,2024-04-25 22:53:00,False,True,False,False,False,False,15.8,8.6,72.73,31,110,37
10519,N,2024-04-26 04:54:00,False,True,False,False,False,False,14,6.8,72.53,23,120,23
10520,N,2024-04-26 14:53:00,False,True,False,False,False,False,17.6,1.4,48.39,9,100,9


In [178]:
mwobs_init.drop(mwobs_init[mwobs_init['relh'] == 'M'].index, inplace=True)
mwobs_init

,WindDir,valid,UP,M,BL,FG,RA,SN,tmpf,dwpf,relh,sknt,vsby,gust
0,W,2014-07-01 04:49:00,False,False,False,True,False,False,55.4,55.4,100,28,0,28
1,SW,2014-07-01 22:57:00,False,True,False,False,False,False,62.6,55.4,77.29,37,10,41
2,W,2014-07-02 04:50:00,False,False,False,True,False,False,57.2,57.2,100,42,0,49
3,W,2014-07-02 14:52:00,False,True,False,False,False,False,60.8,57.2,87.92,21,40,21
4,SW,2014-07-02 22:54:00,False,True,False,False,False,False,57.2,53.6,87.74,20,50,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10517,NW,2024-04-25 14:48:00,False,True,False,False,False,False,10.4,-4,51.41,39,100,45
10518,NW,2024-04-25 22:53:00,False,True,False,False,False,False,15.8,8.6,72.73,31,110,37
10519,N,2024-04-26 04:54:00,False,True,False,False,False,False,14,6.8,72.53,23,120,23
10520,N,2024-04-26 14:53:00,False,True,False,False,False,False,17.6,1.4,48.39,9,100,9


In [180]:
mwobs_init.reset_index(drop = True, inplace=True)

In [182]:
mwobs_init.to_csv('cleaned_mwobs.csv')

In [68]:
delta_t = np.array(mwobs_init['tmpf'][6:-1]).astype(float) - np.array(mwobs_init['tmpf'][0:-7]).astype(float)

In [ ]:
delta_dpt = np.array(mwobs_init['dwpf'][6:-1]).astype(float) - np.array(mwobs_init['dwpf'][0:-7]).astype(float)

In [ ]:
delta_rh = np.array(mwobs_init['relh'][6:-1]).astype(float) - np.array(mwobs_init['relh'][0:-7]).astype(float)

In [43]:
mwobs_init['tmpf'][6:-1] - mwobs_init['tmpf'][0:-7]

TypeError: unsupported operand type(s) for -: 'str' and 'str'